# Черепнина Мария
## Проверка гипотез

In [1]:
import numpy as np
import math

Выбираем гиперпарамептры:

In [2]:
session_set_size = 100
t_threshold = 1.96
N_ASL_samples = 1000
ASL_sample_size = 100

Загружаем данные сплитов:

In [3]:
def load_data(path, line_length=10):
    data = []
    with open(path) as file:
        for line in file:
            data += [np.array([int(i) for i in line[:-1].split(' ')], dtype=int)]
    return np.array(data)
    
split_1 = load_data("c1.txt")
split_2 = load_data("c2.txt")

Считаем CTR (Click-through rate) - отношение числа кликов к числу показов:

In [4]:
def get_CTR(data):
    CTR = np.empty(data.shape[0] // session_set_size)
    for i in range(CTR.size):
        click = np.sum(data[i * session_set_size: (i + 1) * session_set_size], axis=1)
        CTR[i] = np.count_nonzero(click != 0)  / session_set_size
    return CTR

CTR_1 = get_CTR(split_1)
CTR_2 = get_CTR(split_2)

Счиатем среднее кол-во кликов:

In [5]:
def get_AC(data):
    AC = np.empty(data.shape[0] // session_set_size)
    for i in range(AC.size):
        click_count = np.sum(data[i * session_set_size: (i + 1) * session_set_size], axis=1)
        AC[i] = np.sum(click_count) / session_set_size
    return AC

AC_1 = get_AC(split_1)
AC_2 = get_AC(split_2)

Считаем ASL (achieved significance level):

In [6]:
def get_ASL(metric_1, metric_2):
    count = 0
    idx = range(metric_1.size)
    for i in range(N_ASL_samples):
        samples_idx = np.random.choice(idx, size=ASL_sample_size, replace=True)
        mean_1, mean_2 = np.mean(metric_1[samples_idx]), np.mean(metric_2[samples_idx])
        variance_1, variance_2 = np.var(metric_1[samples_idx], ddof=1), np.var(metric_2[samples_idx], ddof=1)
        t = (mean_1 - mean_2) / math.sqrt((variance_1 + variance_2) / N_ASL_samples)
        count += t > t_threshold
    return count / N_ASL_samples

print("CTR ASL: {}".format(get_ASL(CTR_1, CTR_2)))
print("AC ASL: {}".format(get_ASL(AC_1, AC_2)))

CTR ASL: 1.0
AC ASL: 1.0


CTR ASL ≈ 1.0 => гипотеза H_0: CTR на двух сплитах можно объяснить шумами - неверна.

AC ASL ≈ 1.0 => гипотеза H_0: AC на двух сплитах можно объяснить шумами - неверна.